### Global Code and Functions
Run this first to import modules and global functions

In [ ]:
#Modules
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
from IPython.display import Image, display
import re
import math
import torch

# Models
from scipy.spatial import distance
from sklearn.neighbors import NearestNeighbors
from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score
from transformers import CLIPProcessor, CLIPModel
from sklearn.metrics.pairwise import cosine_similarity


# Global Functions

def get_top50_ann(target_embedding, embeddings):
    nn = NearestNeighbors(n_neighbors=51, metric='cosine', algorithm='brute')
    nn.fit(embeddings)
    distances, indices = nn.kneighbors([target_embedding])
    return indices[0][1:]  # Skip the first index because it's the target itself

def get_top50_euclidean(target_embedding, embeddings):
    distances = [distance.euclidean(target_embedding, emb) for emb in embeddings]
    indices = np.argsort(distances)[1:51]  # Skip the first index because it's the target itself
    return indices

def display_image(index):
    display(Image(filename=f"thumbnails_folder2large/{g_movie_embeddings[index]['input']}"))

def display_images(indices, embeddings):
    fig, axes = plt.subplots(10, 10, figsize=(20, 10))
    for i, ax in enumerate(axes.flat):
        ax.imshow(plt.imread(f"thumbnails_folder2large/{g_movie_embeddings[indices[i]]['input']}"))
        ax.axis('off')
    plt.show()

def display_images_first_x_last_x(indices, first_x, last_x, cluster_n=0):
    # Select the first_x and last_x indices
    selected_indices = indices[:first_x] + indices[-last_x:]
    
    # Calculate the number of rows and columns for the subplot
    total_images = first_x + last_x
    cols = 10
    rows = math.ceil(total_images / cols)
    
    fig, axes = plt.subplots(rows, cols, figsize=(20, 2 * rows))
    axes = axes.ravel()  # Flatten the axes array
    
    # Hide all axes
    for ax in axes:
        ax.axis('off')
    
    # Display images on the first len(selected_indices) axes
    for i, idx in enumerate(selected_indices):
        axes[i].imshow(plt.imread(f"thumbnails_folder2large/{g_movie_embeddings[idx]['input']}"))
        axes[i].axis('on')
    
    plt.tight_layout()
    plt.title(f"Cluster {cluster_n} -  First {first_x} and Last {last_x} Images - Total Images in Cluster: {len(indices)}")
    plt.show()

def display_cluster_images(cluster_labels, cluster_number):
    # Get indices of images in the cluster
    indices = [i for i, label in enumerate(cluster_labels) if label == cluster_number]
    
    # Display images
    display_images(indices)

def display_cluster_images_first_last_x(cluster_labels, cluster_number, first_x, last_x):
    # Get indices of images in the cluster
    indices = [i for i, label in enumerate(cluster_labels) if label == cluster_number]
    
    # Display images
    display_images_first_x_last_x(indices, first_x, last_x, cluster_number)

def find_and_remove_intro_and_subtitles(g_only_embeddings, threshold=0.7):
    # Load the CLIP model and processor
    model = CLIPModel.from_pretrained("openai/clip-vit-large-patch14")
    processor = CLIPProcessor.from_pretrained("openai/clip-vit-large-patch14")

    # Get the text embeddings for the intro and subtitles
    inputs = processor(text=["Image of Walt Disney Movie Intro", "Walt Disney Movie Intro", "Movie closing credits", "Movie end credits", "Image with lots of closing credits"], return_tensors="pt", padding=True)
    text_embeddings = model.get_text_features(**inputs)
    text_embeddings_np = text_embeddings.detach().numpy()

    # Calculate the cosine similarity between the text embeddings and the movie embeddings
    similarities = cosine_similarity(text_embeddings_np, g_only_embeddings)

    # Find the indices of the embeddings that are similar to the intro and subtitles
    intro_subtitle_indices = np.where(similarities.max(axis=0) > threshold)[0]

    print("Number of images(Intro and Closing credits) to remove:", intro_subtitle_indices)

    # Remove the intro and subtitles from the movie embeddings
    for idx in sorted(intro_subtitle_indices, reverse=True):
        del g_movie_embeddings[idx]
        g_only_embeddings = np.delete(g_only_embeddings, idx, axis=0)
    

# Global Variables

g_movie_embeddings = json.load(open("honey_i_shrunk_the_kids_movie_embeddings_1_second.json"))
g_only_embeddings = np.array([emb['embedding'] for emb in g_movie_embeddings])
find_and_remove_intro_and_subtitles(g_only_embeddings, threshold=0.237)


### Cluster Analysis

We will compare clustering with t-SNE (t-Distributed Stochastic Neighbor Embedding) and PCA (Principal Component Analysis) dimensionality reduction algorithms.

#### t-SNE

In [ ]:
# Using t-SNE to embed the vectors into 2D
tsne = TSNE(n_components=2, random_state=42)
tSNE_embedded_vectors = tsne.fit_transform(g_only_embeddings)


#### PCA

In [ ]:
# Using PCA to embed the vectors into 2D
pca = PCA(n_components=2)
PCA_embedded_vectors = pca.fit_transform(g_only_embeddings)

#### Cluster t-SNE and PCA with K-Means and display Silhoutte Score

**Silhouette Score**: Measures how similar an object is to its own cluster compared to other clusters. The score ranges from -1 to 1, where a high value indicates that the object is well matched to its own cluster and poorly matched to neighboring clusters.

Lets find the best clustering number for t-SNE and PCA...

In [ ]:
X = tSNE_embedded_vectors

# Range of clusters to try
num_clusters = range(2, 20)

# List to hold silhouette scores
sil_scores = []

# Loop over number of clusters
for k in num_clusters:
    # Perform clustering
    kmeans = KMeans(n_init="auto", n_clusters=k, random_state=42).fit(X)
    
    # Get cluster labels
    labels = kmeans.labels_
    
    # Compute silhouette score and append to list
    sil_score = silhouette_score(X, labels)
    sil_scores.append(sil_score)

# Plot silhouette scores
plt.plot(num_clusters, sil_scores, 'bx-')
plt.title('t-SNE')
plt.xlabel('k (number of clusters)')
plt.ylabel('Silhouette Score')
plt.show()



X = PCA_embedded_vectors

# Range of clusters to try
num_clusters = range(2, 20)

# List to hold silhouette scores
sil_scores = []

# Loop over number of clusters
for k in num_clusters:
    # Perform clustering
    kmeans = KMeans(n_init="auto", n_clusters=k, random_state=42).fit(X)
    
    # Get cluster labels
    labels = kmeans.labels_
    
    # Compute silhouette score and append to list
    sil_score = silhouette_score(X, labels)
    sil_scores.append(sil_score)

# Plot silhouette scores
plt.plot(num_clusters, sil_scores, 'bx-')
plt.title('PCA')
plt.xlabel('k (number of clusters)')
plt.ylabel('Silhouette Score')
plt.show()

Lets cluster with t-SNE and PCA best Silhoutte Scores.

In [ ]:
# Performing KMeans clustering with best k silhoutte score.
kmeans = KMeans(n_init="auto", n_clusters=16, random_state=42)
tSNE_clusters = kmeans.fit_predict(tSNE_embedded_vectors)

kmeans = KMeans(n_init="auto", n_clusters=4, random_state=42)
PCA_clusters = kmeans.fit_predict(PCA_embedded_vectors)


# Extracting numbers from file names for labels
labels = [re.search(r'\d+', vector['input']).group() for vector in g_movie_embeddings]

#t-SNE
# Plotting the embedded vectors with cluster coloring
sns.set_theme()
plt.figure(figsize=(12, 8))  # Adjust the figure size as needed
sns.scatterplot(x=tSNE_embedded_vectors[:, 0], y=tSNE_embedded_vectors[:, 1], hue=tSNE_clusters, palette='bright', legend='full', s=100)
for i, vec in enumerate(tSNE_embedded_vectors):
    plt.text(vec[0] + 0.02, vec[1] + 0.02, labels[i], fontsize=6)  # Adding labels
plt.xlabel('Dimension 1')
plt.ylabel('Dimension 2')
plt.title('t-SNE Embedded Vectors with KMeans Clustering (k=16)')
plt.legend(title='Cluster')
plt.show()

#PCA
# Plotting the embedded vectors with cluster coloring
sns.set_theme()
plt.figure(figsize=(12, 8))  # Adjust the figure size as needed
sns.scatterplot(x=PCA_embedded_vectors[:, 0], y=PCA_embedded_vectors[:, 1], hue=PCA_clusters, palette='bright', legend='full', s=100)
for i, vec in enumerate(PCA_embedded_vectors):
    plt.text(vec[0] + 0.02, vec[1] + 0.02, labels[i], fontsize=6)  # Adding labels
plt.xlabel('Dimension 1')
plt.ylabel('Dimension 2')
plt.title('PCA Embedded Vectors with KMeans Clustering (k=4)')
plt.legend(title='Cluster')
plt.show()


#### Let sample images in t-SNE clusters

In [ ]:
unique_clusters = set(tSNE_clusters)

for cluster in unique_clusters:
    display_cluster_images_first_last_x(tSNE_clusters, cluster, 10, 10)

#### Lets sample images in PCA clusters

In [ ]:
unique_clusters = set(PCA_clusters)

for cluster in unique_clusters:
    display_cluster_images_first_last_x(PCA_clusters, cluster, 10, 10)

In [ ]:
display_cluster_images_first_last_x(tSNE_clusters, cluster, 192, 100)

In [ ]:
# Convert labels to timestamps by dividing by frame rate
# Assuming `frame_rate` is the frame rate of the movie
frame_rate = 24
timestamps = [int(label) / frame_rate for label in labels]

# Create a timeline plot for the t-SNE clusters
plt.figure(figsize=(12, 6))
plt.scatter(timestamps, tSNE_clusters, c=tSNE_clusters, cmap='viridis')
plt.xlabel('Time')
plt.ylabel('Cluster')
plt.title('t-SNE Clusters Over Time')
plt.colorbar(label='Cluster')
plt.show()

# Create a timeline plot for the PCA clusters
plt.figure(figsize=(12, 6))
plt.scatter(timestamps, PCA_clusters, c=PCA_clusters, cmap='viridis')
plt.xlabel('Time')
plt.ylabel('Cluster')
plt.title('PCA Clusters Over Time')
plt.colorbar(label='Cluster')
plt.show()

### Film Description
#### Honey, I Shrunk the Kids
1989 PG 1h 33m

From the IMDB website:

"The scientist father of a teenage girl and boy accidentally shrinks his and two other neighborhood teens to the size of insects. Now the teens must fight diminutive dangers as the father searches for them."

IMDB website. (n.d.). imdb.com. Retrieved April 27, 2024, from https://www.imdb.com/title/tt0097523/

In 'Honey, I Shrunk the Kids,' an eccentric inventor, Wayne Szalinski, accidentally shrinks his and his neighbor's children with his experimental shrink ray. The miniature kids must navigate a perilous journey across their now-gigantic backyard, encountering obstacles like insects and sprinklers, as they try to return home.

The film is notable for its creative visual effects that magnify ordinary environments into epic landscapes. It's a blend of adventure, humor, and family dynamics, ultimately showcasing the children's resourcefulness and the parents' determination to rescue their kids. The movie was a commercial success and spawned a franchise including sequels and a television series.

### Methods Summary

This section should highlight methods you used in your exploratory analysis. You should include at least one clustering technique or develop another way to relate frames to other frames. You should also consider dimensionality reduction.

Each thumbnail is one frame of the Movie. Each thumbnail has been analyzed and embedded via CLIP. Referenced Model: https://replicate.com/andreasjansson/clip-features/examples .

#### For hypothesis 1 we used the following methods

1. Explore CLIP through a natural query for objects and investigate the surounding frames.
2. Look at the Euclidean distance of similar frames.
3. Explore similar frames for insights.

### Hunches and Hypotheses

This section should summarize the questions that you asked about the film that could potentially be answered by exploratory analysis. You should ask at least three questions.

**Hypothesis 1** 
-   Using Clip to find an object and observe ~5 frames before and after and interpret the results.
-   A hypothesis we discussed was that certain characters, or objects in the movie would be used for dramatic or comedic effect. We are using CLIP to identify these objects and then explore the surrounding frames to discover any evidence to support or refute this hypothesis.

**Hypothesis 2** 
-   Scene Consistency and Transition - Frames that are visually and thematically similar cluster together tightly in t-SNE and PCA visualizations, and distinct clusters correspond to different scenes or settings in the movie.
-  **Rationale:** This hypothesis tests the ability of CLIP embeddings, which capture both visual and semantic content, to differentiate between distinct scenes based on their visual content and thematic elements.

**Hypothesis 3**
-   Find a model that can analyze an image and identify how many objects are there. 2 trees, 1 dog, 3 humans, a house...etc

### Results and Interpretation

**Hypothesis 1:**
- Begin by looking for the top images of a dog, lawnmower and an ant returned from CLIP queries.
- Explore the frames surrounding the images identified by the above queries.

These are the top matches returned by CLIP.

1. 'a photo of a white and brown dog' = thumbnail_5039.jpg
2. 'a photo of a lawnmower' = thumbnail_4734.jpg
3. 'a photo of an ant' = thumbnail_3231.jpg

The indexes we are interested in are 5038, 4733 and 3230.

- For the dog image:

From this sequence of frames, we see the children running, the parents talking, the dog looking up at the table, the kids waving their arms for attention and ends with the father narrowly avoiding accidentally eating his shrunken son. The dog being in the center anchors the dramatic tension of the scene. He almost looks like he is trying to warn the father or somehow knows more than the humans. Sentient animals with superior knowledge or wisdom than humans has been a trope used in several other films which depict animals (especially in comedies) as central figures.

- For the lawnmower image:

In this scene a neighbor kid is remotely controlling the lawnmower while the children are in the grass. This is a clear action scene with a threat elliciting dramatic flight.

There doesn't seem to be many similar frames in the movie for the lawnmower. This is probably because it is a singular threat evnt used for dramatic tension in a single scene. It does look like it may appear around 790, so let's see if that is foreshadowing.

In the scene around index 790 we see the son showing the remote controlled lawnmower to the boy who appears later controlling the mower in the flight scene. So this is an example of foreshadowing in a film that we identified by using the data to point us to something interesting!

- For the ant image:

In this scene, we see a boy easting some white substance, followed by a girl who appears scared or surprised. The next frame shows the cause, which is an ant menacing the kids, which is noticed by the boy as well, and the kids seem to begin fleeing the threat.

The ant seems to be a later threat in the film, possibly also foreshadowed around 2180. However it appears to show up for an extended time around 4400.

2180 looks like it matches the ant scene because of the setting. They both have extensive vegetation in the scene, which probably accounts for the lower distance than the later match around 4400.

_**This identified the same scene that we discovered through exploration.**_

We began with a hypothesis that the dog would be comedic relief and the ant and lawnmower would be for dramatic effect or threat of danger.We used CLIP to identify the inital frames of interest, then proceeded analyzing the target frames using the Euclidian distance, performing more exploration of similar frames and then circling back to a more specific CLIP question. The original hypothesis that these objects would be used as comedic relief or dramatic seems to have some justification, as the lawnmower and ant appear intially as dramatic threats, but from this discovery, it looks like the ant actually fought a scorpion and saved the kids, which indicated it was also used to elicit a sympathetic response from the audience. 

**Hypothesis 2:** 
- After removing closing credits frames and applying t-SNE and K-means we examined samples from identified clusters and we can observe that frames correspond to specific scenes or types of scenes (indoor vs outdoor, calm vs action-packed). Embeddings effectively capture scene-specific features and can be used to segment the movie based on visual content. Also, clusters seem to reflect the proper timeframe and transition of the movie.




### Reflection

Reflect on your process of analysis. What worked well and did not work well? Describe the limitations of the work and describe what you would work on with more time.

We initially began exploring CLIP to discover the capabilities it provided. We were impressed with the model's ability to return results from simple english questions, so we brainstormed for some ideas that we could explore it as well as other tools to compare and contrast CLIP to other tools and also to discover what it could tell us about the film itself.

We came up with the hypothesis that we could identify some objects in the film and explore whether they provided an assumed purpose in the movie. We did achieve (subjectively) a confirmation tthat the objects were used in a way that coincided with our hypothesis. What was interesting is the way that our initial exploration led to a secondary use that effectively used CLIP to identify a further area of research. This resulted in the surprising find that the ant seems to not only be used as a threat but also as an ally to evoke both fear and empathy.


### Hypothesis 1 Code and Results in Detail

#### Define helper functions for exploration

In [ ]:
dog_idx = 5038
mower_idx = 4733
ant_idx = 3230
image_root = 'thumbnails_folder2large/'

model = CLIPModel.from_pretrained("openai/clip-vit-large-patch14")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-large-patch14")

with open("honey_i_shrunk_the_kids_movie_embeddings_1_second.json", 'r') as file:
    movie_embeddings = json.load(file)

def euclidean_distance(array1, array2):
    # Convert the arrays to NumPy arrays
    array1_np = np.array(array1)
    array2_np = np.array(array2)
    
    # Calculate the Euclidean distance
    distance = np.linalg.norm(array1_np - array2_np)
    return distance

def find_and_display_matches(text_queries, top_k=5):
    inputs = processor(text=text_queries, return_tensors="pt", padding=True)
    text_embeddings = model.get_text_features(**inputs)
    text_embeddings_np = text_embeddings.detach().numpy()
    movie_embeddings_np = np.array([movie['embedding'] for movie in movie_embeddings])
    similarities = cosine_similarity(text_embeddings_np, movie_embeddings_np)
    for index, text_query in enumerate(text_queries):
        print(f"Top matches for: {text_query}")
        top_indices = np.argsort(similarities[index])[::-1][:top_k]
        for i in top_indices:
            frame = movie_embeddings[i]['input']
            print(f"Displaying frame: {frame}")
            display(Image(filename=f'thumbnails_folder2large/{frame}'))

def plot_euclidean_distance_from(target_idx):
    target = movie_embeddings[target_idx]
    image_path = image_root+target["input"]
    # Display the image
    display(Image(filename=image_path))

    index_to_distance = []

    # Iterate through the input list
    for emb in movie_embeddings:
        current_dist = euclidean_distance(emb["embedding"], target["embedding"])
        index_to_distance.append(current_dist)

    # Create a plot using Seaborn
    sns.set(style="whitegrid")  # Set the style
    plt.figure(figsize=(10, 6))  # Set the figure size
    sns.lineplot(x=range(len(index_to_distance)), y=index_to_distance)  # Plot the array with index as x-axis
    plt.xlabel("Index")  # Set the x-axis label
    plt.ylabel("Distance")  # Set the y-axis label
    plt.title("Distance from Target Over Film")  # Set the title
    plt.show()  # Show the plot

def display_surrounding_frames(target_idx, frame_range=5):
    idx_begin = target_idx - frame_range
    idx_end = target_idx + frame_range + 1

    display_frames = movie_embeddings[idx_begin:idx_end]
    for emb in display_frames:
        print(f'frame {emb["input"]}')
        image_path = image_root + emb["input"]
        display(Image(filename=image_path))



#### Exploring the Dog
Dislpay the images

In [ ]:
display_surrounding_frames(dog_idx)

In [ ]:
plot_euclidean_distance_from(dog_idx)

In [ ]:
# Using the following to explore 500, 2100 and 3400
#display_surrounding_frames(3400, frame_range=40)
# Discovered the dog in
# thumbnail_0514.jpg
# thumbnail_2078.jpg
# thumbnail_2131.jpg
# thumbnail_3427.jpg
# Indexes are are 0 based
display_surrounding_frames(513, frame_range=2)

The dog is watching the father's scientific experiment.

In [ ]:
display_surrounding_frames(2077, frame_range=2)

The dog seems to notice something outside.

In [ ]:
display_surrounding_frames(2130, frame_range=2)

The parents are distracted, and the dog wants to investigate what is going on outside, once again exhibiting a higher sense of awareness than the humans.

In [ ]:
display_surrounding_frames(3426, frame_range=2)

Here the dog is spinning the father around and disrupting his search for the children, which is an example of a comedic scene.

#### Exploring the Lawnmower
Dislpay the images

In [ ]:
display_surrounding_frames(mower_idx)

In [ ]:
plot_euclidean_distance_from(mower_idx)

In [ ]:
# Display around 790
#display_surrounding_frames(790, frame_range=40)
display_surrounding_frames(790, frame_range=5)

#### Exploring the Ant
Dislpay the images

In [ ]:
display_surrounding_frames(ant_idx)

In [ ]:
plot_euclidean_distance_from(ant_idx)

In [ ]:
# Display around 2180
#display_surrounding_frames(2180, frame_range=40)
display_surrounding_frames(2180, frame_range=1)

In [ ]:
# Display around 4400
#display_surrounding_frames(4400, frame_range=40)
display_surrounding_frames(4435, frame_range=1)

The scene around 4400 shows an ant fighting a scorpion and appearing to save the kids.

Use CLIP to ask about 'a photo of an ant fighting a scorpion' to see what it returns.

In [ ]:
exploration_query = ['a photo of an ant fighting a scorpion'] 
find_and_display_matches(exploration_query, top_k=10)